## preparation

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize
import random

In [2]:
fares = pd.read_csv('Fares.csv', sep = ';')
flights = pd.read_csv('Flights.csv', sep = ';')


In [3]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')

In [4]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')
history['id'] = '001'
for i in range(2,101,1):
    if len(str(i)) == 1:
        i = '00' + str(i)
    elif len(str(i)) == 2:
        i = '0' + str(i)
    else:
        i = str(i)
        
        
    hist = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_' + i + '.csv', sep = ';')
    hist['id'] = i
    history = history.append(hist)
    
    
   

In [122]:
def get_demands(fare_id, history):
    filt_history = history.loc[history.FARE_ID == fare_id,]
    demands = filt_history['id'].value_counts()
    return demands

def get_capacity(flight_id):
    return flights.loc[flights.FLIGHT_ID == flight_id]['CAPACITY']
    
def get_price(fare_id):
    return fares.loc[fares.FARE_ID == fare_id]['PRICE']

def get_flights(fare_id):
    dep_air  = int(fares.loc[fares.FARE_ID == fare_id]['DEPARTURE_AIRPORT'])
    dest_air = int(fares.loc[fares.FARE_ID == fare_id]['DESTINATION_AIRPORT'])
   
    if dep_air == 1 or dest_air == 1:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air) &
                           (flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]
    else:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air)]['FLIGHT_ID'], 
                flights.loc[(flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]
    
def get_shadow_price(fare_id, shadow_prices_df):
    flights_needed = get_flights(fare_id)
    if len(flights_needed) == 1:
        shadow_price = shadow_prices_df[flights_needed[0] -1]
    else:
        shadow_price = shadow_prices_df[flights_needed[0]-1] + shadow_prices_df[flights_needed[1]-1]
        
    return(shadow_price)
    
    
def allocate_seat(allocations, shadow_prices_df, fare_id, sold_tickets, flight_fare_df, bucket_flights):
    #is there allocation available?
    if allocations[fare_id-1] > 0:
        allocations[fare_id-1] -= 1
        sold_tickets[fare_id-1] += 1 
    else :
        shadow_price = get_shadow_price(fare_id, shadow_prices_df)
        flights_needed = get_flights(fare_id)
#         flights_needed = get_flights(fare_id)
#         if len(flights_needed) == 1:
#             shadow_price = shadow_prices_df[flights_needed-1]
#         else:
#             shadow_price = shadow_prices_df[flights_needed[0]-1] + shadow_prices_df[flights_needed[1]-1]
      
        if len(flights_needed) == 1:
            if flights_needed in bucket_flights:
                bucket_flights = np.delete(bucket_flights,np.where(bucket_flights == flights_needed)[0])
                sold_tickets[fare_id-1] +=1
            else:
                candidate_fares = np.where(flight_fare_df.iloc[int(flights_needed[0]) - 1,] == 1)[0]
                fully_booked_fares = np.array(np.where(allocations == 0)) + 1
                                                
                candidate_fares = candidate_fares[~np.in1d(candidate_fares,fully_booked_fares)]
                print(candidate_fares)
                
                
                candidate_fare_shadow_prices = np.zeros((candidate_fares.shape))
                for i in range(candidate_fares.shape[0]):
                    candidate_fare_shadow_prices[i] = get_shadow_price(candidate_fares[i]+1, shadow_prices_df)
                
                try:
                    sacrificed_fare = \
                    candidate_fares[np.where(candidate_fare_shadow_prices == \
                                             candidate_fare_shadow_prices.min())[0][0]]
                except:
                    break
                
                
                allocations[sacrificed_fare-1] -= 1
                sold_tickets[fare_id-1] += 1
                
                if sum(flight_fare_df.iloc[:,sacrificed_fare-1]) == 2:
                    sf_flights = np.array(np.where(flight_fare_df.iloc[:,sacrificed_fare-1] == 1)) +1
                    sf_flights = np.delete(sf_flights, sf_flights == flights_needed)
                    bucket_flights = np.append(bucket_flights, sf_flights)
                
        else:
            if np.array(flights_needed[0]) in bucket_flights:
                bucket_flights = np.delete(bucket_flights,np.where(bucket_flights == flights_needed)[0])
                sold_tickets[fare_id-1] +=1
            else:
                candidate_fares = np.where(flight_fare_df.iloc[int(flights_needed[0]) - 1,] == 1)[0]
                print(candidate_fares)
                fully_booked_fares = np.array(np.where(allocations == 0)) + 1
                candidate_fares = candidate_fares[~np.in1d(candidate_fares,fully_booked_fares)]
                
                candidate_fare_shadow_prices = np.zeros((candidate_fares.shape))
                for i in range(candidate_fares.shape[0]):
                    candidate_fare_shadow_prices[i] = get_shadow_price(candidate_fares[i] +1, shadow_prices_df)
                
                try:
                    sacrificed_fare = \
                    candidate_fares[np.where(candidate_fare_shadow_prices == \
                                             candidate_fare_shadow_prices.min())[0][0]]
                except:
                    break
                
                allocations[sacrificed_fare-1] -= 1
                sold_tickets[fare_id-1] += 1
                
                if sum(flight_fare_df.iloc[:,sacrificed_fare-1]) == 2:
                    sf_flights = np.array(np.where(flight_fare_df.iloc[:,sacrificed_fare-1] == 1)) + 1
                    sf_flights = np.delete(sf_flights, sf_flights == flights_needed)
                    bucket_flights = np.append(bucket_flights, sf_flights)
                
            if np.array(flights_needed[1]) in bucket_flights:
                bucket_flights = np.delete(bucket_flights,np.where(bucket_flights == flights_needed)[0])
                sold_tickets[fare_id-1] +=1
            else:
                candidate_fares = np.where(flight_fare_df.iloc[int(flights_needed[0]) - 1,] == 1)[0]
                print(candidate_fares)
                fully_booked_fares = np.array(np.where(allocations == 0)) + 1
                candidate_fares = candidate_fares[~np.in1d(candidate_fares,fully_booked_fares)]
                print(candidate_fares)
                
                candidate_fare_shadow_prices = np.zeros((candidate_fares.shape))
                for i in range(candidate_fares.shape[0]):
                    candidate_fare_shadow_prices[i] = get_shadow_price(candidate_fares[i] +1, shadow_prices_df)
                
                try:
                    sacrificed_fare = \
                    candidate_fares[np.where(candidate_fare_shadow_prices == \
                                             candidate_fare_shadow_prices.min())[0][0]]
                except:
                    break
                
                allocations[sacrificed_fare - 1] -= 1
                sold_tickets[fare_id-1] += 1
                
                if sum(flight_fare_df.iloc[:,sacrificed_fare-1]) == 2:
                    sf_flights = np.array(np.where(flight_fare_df.iloc[:,sacrificed_fare-1] == 1)) + 1
                    sf_flights = np.delete(sf_flights, sf_flights == flights_needed)
                    bucket_flights = np.append(bucket_flights, sf_flights)
            
            #napocitat ktery fares maj volny alokace, porovnat shadow pricy, vzit jim je, priradit a zbytek do bucketu
            
        
        
#     return allocations, shadow_prices_df, fare_id, sold_tickets, bucket_flights  
    
def run_allocations(history_day, allocations, shadow_prices_calc, flight_fare_df):
    sold_tickets = np.zeros((144,1))
    bucket_flights = np.array([])
    for i in range(history_day.shape[0]):
        allocate_seat(allocations, shadow_prices_calc, history_day['FARE_ID'][i], sold_tickets, flight_fare_df, bucket_flights)
        if i % 50 == 0:
            print(i)
    
    return sold_tickets
    




    

In [117]:
flights_needed = get_flights(19)
flight_fare_df

0    1    2    3    4    5    6    7    8    9   ...   134  135  136  137  \
0     0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
1     0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
2     0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
3     0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
4     0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
5     0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
6     0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
7     0    0    0    0    0    0    0    0    0    0 ...     1    1    1    1   
8     1    1    0    0    0    0    0    0    0    0 ...     0    0    0    0   
9     0    0    1    1    0    0    0    0    0    0 ...     0    0    0    0   
10    0    0    0    0    1    1    0    0    0    0 ...     1    1    0    0   
11    0    0    0    0    0    0    1    1    0    0 ...     0    0    1    1   
12    0    0    0    0    0    0    0    0    1    1 ...     0    0    0    0   
13    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
14    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
15    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   

    138  139  140  141  142  143  
0     0    0    0    0    0    0  
1     0    0    0    0    0    0  
2     0    0    0    0    0    0  
3     0    0    0    0    0    0  
4     0    0    0    0    0    0  
5     0    0    0    0    0    0  
6     0    0    0    0    0    0  
7     1    1    1    1    1    1  
8     0    0    0    0    0    0  
9     0    0    0    0    0    0  
10    0    0    0    0    0    0  
11    0    0    0    0    0    0  
12    1    1    0    0    0    0  
13    0    0    1    1    0    0  
14    0    0    0    0    1    1  
15    0    0    0    0    0    0  

[16 rows x 144 columns]

In [130]:
flight_fare_df = pd.DataFrame(index=range(flights.shape[0]),columns=range(fares.shape[0]), data = 0)
for i in range(fares.shape[0]):
    used_flights = get_flights(i+1)
    try: 
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        flight_fare_df.iloc[int(used_flights[1]) - 1, i] = 1
    except:
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1

exp_demand = np.array(history['FARE_ID'].value_counts().sort_index()) / 100
bounds = [(0, exp_demand[i]) for i in range(len(exp_demand))]
capacity = flights['CAPACITY']

allocations = scipy.optimize.linprog(c= - np.array(fares['PRICE']), A_ub=np.array(flight_fare_df), 
                                     b_ub=capacity, bounds=bounds).x

allocations = np.floor(allocations)
print(sum(allocations))
shadow_prices = scipy.optimize.linprog(c=capacity.reshape((16,)), A_ub= - np.array(flight_fare_df).T,
                                       b_ub= - np.array(fares['PRICE']).T).x

102.0


C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


NameError: name 'candidate_fares' is not defined

In [131]:
hist = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\Future\Bookings_001.csv', sep = ';')
tickets = run_allocations(hist, allocations, shadow_prices, flight_fare_df)

0
[128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
[128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
[131 135 141 142]
[80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]
[80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]
[85 86 88 93 94]
[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
[49 52 53 56 61 62]
[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
[52 53 56 61 62]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]

C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: FutureWarning: in the future insert will treat boolean arrays and array-likes as boolean index instead of casting it to integer
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:119: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:119: FutureWarning: in the future insert will treat boolean arrays and array-likes as boolean index instead of casting it to integer



[22 24 27 28 29 30]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
[22 24 27 28 29 30]
[ 24  41  56 123]
[128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
[128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
[135 142]
[ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111]
[ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111]
[ 97  98 102 105 110 111]
[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
[33 34 37 39 41]
[ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111]
[ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111]

C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: FutureWarning: in the future insert will treat boolean arrays and array-likes as boolean index instead of casting it to integer
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.



[102 105 110 111]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
[128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
[128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
[]


ValueError: zero-size array to reduction operation minimum which has no identity

In [129]:
np.min(np.array([]))

ValueError: zero-size array to reduction operation minimum which has no identity